<img src='https://media.istockphoto.com/vectors/banner43-vector-id944254822?b=1&k=6&m=944254822&s=612x612&w=0&h=tR0tpZI72oRw87G9fzN4BFV1hT9B-RC_t0WPpO9b7Jg=' style="width:200px;height:200px;">


As of now , only genres and movie-casts(3) are used to find similar movies.<br>
Next Tasks:-
* Add ratings, release year and director to filter similar movies.
* Increase the movie dataset to include ohter 30k movies. 

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
from scipy import spatial
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('FinalData.csv')

In [4]:
df.reset_index(drop=True,inplace=True)

In [5]:
df.head(4)

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,combined
0,insidious: the last key,adam robitel,lin shaye,angus sampson,leigh whannell,mystery horror thriller,adam robitel lin shaye angus sampson leigh wha...
1,the strange ones,lauren wolkstein,alex pettyfer,james freedson-jackson,emily althaus,thriller drama,lauren wolkstein alex pettyfer james freedson-...
2,stratton,simon west,dominic cooper,austin stowell,gemma chan,action thriller,simon west dominic cooper austin stowell gemma...
3,sweet country,warwick thornton,bryan brown,sam neill,,drama history western,warwick thornton bryan brown sam neill drama...


In [6]:
df.drop('combined',axis=1,inplace=True)

In [7]:
g = []
for i in df.genres:
    w = []
    w = i.split(' ')
    for j in w:
        if j not in g:
            g.append(j)
print(g)
g_list = g[:-1]
g_list.sort()
g_list.append('no genres listed')
print(g_list)

['mystery', 'horror', 'thriller', 'drama', 'action', 'history', 'western', 'crime', 'comedy', 'fantasy', 'adventure', 'animation', 'war', 'romance', 'music', 'science', 'fiction', 'family', 'documentary', 'sci-fi', 'musical', 'biography', 'sport', 'foreign', '']
['action', 'adventure', 'animation', 'biography', 'comedy', 'crime', 'documentary', 'drama', 'family', 'fantasy', 'fiction', 'foreign', 'history', 'horror', 'music', 'musical', 'mystery', 'romance', 'sci-fi', 'science', 'sport', 'thriller', 'war', 'western', 'no genres listed']


In [8]:
def setGenres_Mat(genre):
    movieGenresMatrix = []
    movieGenresList = genre.split(' ')
    for x in g_list:
        if (x in movieGenresList):
            movieGenresMatrix.append(1)
        else:
            movieGenresMatrix.append(0) 
    return movieGenresMatrix
    
df['genresMatrix'] = df.apply(lambda x: np.array(list(setGenres_Mat(x['genres']))), axis=1)

df.head(2)

,movie_title,director_name,actor_1_name,actor_2_name,actor_3_name,genres,genresMatrix
0,insidious: the last key,adam robitel,lin shaye,angus sampson,leigh whannell,mystery horror thriller,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
1,the strange ones,lauren wolkstein,alex pettyfer,james freedson-jackson,emily althaus,thriller drama,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [9]:
df['actors'] = df[df.columns[2:5]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)

In [10]:
dff = df[['movie_title','genresMatrix','actors']]
dff.head(3)

,movie_title,genresMatrix,actors
0,insidious: the last key,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","lin shaye,angus sampson,leigh whannell"
1,the strange ones,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","alex pettyfer,james freedson-jackson,emily alt..."
2,stratton,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","dominic cooper,austin stowell,gemma chan"


In [11]:
actor_1_list = list(df.actor_1_name)

In [12]:
actor_2_list = list(df.actor_2_name)

In [13]:
actor_3_list = list(df.actor_3_name)

In [14]:
a_list = list(set().union(actor_1_list, actor_2_list,actor_3_list))

In [15]:
len(a_list)

11497

In [16]:
def set_actor_mat(actors):
    actor_matrix = []
    actor_list = actors.split(',')
    for i in a_list:
        if (i in actor_list):
            actor_matrix.append(1)
        else:
            actor_matrix.append(0) 
    return actor_matrix
    
dff['actor_matrix'] = dff.apply(lambda x: np.array(list(set_actor_mat(x['actors']))), axis=1)

dff

,movie_title,genresMatrix,actors,actor_matrix
0,insidious: the last key,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","lin shaye,angus sampson,leigh whannell","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,the strange ones,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","alex pettyfer,james freedson-jackson,emily alt...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,stratton,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","dominic cooper,austin stowell,gemma chan","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,sweet country,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, ...","bryan brown,sam neill,","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,the commuter,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","liam neeson,vera farmiga,patrick wilson","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...
11376,an american vampire story,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","trevor lissauer,johnny venocur,adam west","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11377,fit to kill,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","dona speir,roberta vasquez,bruce penhall","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11378,cadet kelly,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","hilary duff,christy carlson romano,gary cole","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
11379,robin hood,"[1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","patrick bergin,uma thurman,david morrissey","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [17]:
dff.drop('actors',axis=1,inplace=True)

In [18]:
dff.head(2)

,movie_title,genresMatrix,actor_matrix
0,insidious: the last key,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,the strange ones,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
dff.reset_index(drop=True,inplace=True)
dff.shape

(11381, 3)

In [20]:
dff['movie_id'] = np.arange(0,dff.shape[0])

In [21]:
dff = dff[['movie_id','movie_title','genresMatrix','actor_matrix']]
dff.head(3)

,movie_id,movie_title,genresMatrix,actor_matrix
0,0,insidious: the last key,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,1,the strange ones,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,2,stratton,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [32]:
genresSimilarityWeight = 0.5
actorsSimilarityWeight = 0.5
titleSimilarityWeight = 1

def checksimi(movie_id):
    basis_genre = np.array(list(dff[dff['movie_id']==movie_id]['genresMatrix']))
    basis_actor = np.array(list(dff[dff['movie_id']==movie_id]['actor_matrix']))
    
#     movies_similar = dff

    dff['similarity'] = dff.apply(lambda x: 
                   spatial.distance.cosine(x['genresMatrix'],basis_genre)*genresSimilarityWeight
                 + spatial.distance.cosine(x['actor_matrix'],basis_actor)*actorsSimilarityWeight,
                        axis=1)
    
    return dff[['movie_id','movie_title','similarity']].sort_values('similarity')

In [33]:
similar_movies = checksimi(3992)

In [34]:
similar_movies[,:]

,movie_id,movie_title,similarity
3992,3992,hulk,0.000000
267,267,alita: battle angel,0.458333
6023,6023,the day the earth stood still,0.458333
2032,2032,the rocketeer,0.497923
8762,8762,supernova,0.500000
